# Convolutional Neural Network for Fashion MNIST

### Setup

In [2]:
# Import statements
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

TensorFlow version: 2.10.0


In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train/255, x_test/255

# Add a channels dimension (Marked for review)
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# Batching and shuffling the dataset (Marked for review)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

2022-10-11 16:58:56.400355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Building the Model

In [4]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(64, (3,3), activation="relu")
        self.maxPooling1 = MaxPooling2D((2,2))
        self.flatten = Flatten()
        self.d1 = Dense(64, activation="relu")
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.maxPooling1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [5]:
# Choosing and optimizer and loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# This is the metrics for loss and accuracy
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")


In [6]:
# Training the Model

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training = True only needed for dropout and other like layers
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [7]:
# Test the model
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [8]:
EPOCHS = 5
for epoch in range(EPOCHS):
    # Reset metrics at the start of each epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels, in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )
    

Epoch 1, Loss: 0.39423954486846924, Accuracy: 85.94667053222656, Test Loss: 0.33697471022605896, Test Accuracy: 87.97000122070312
Epoch 2, Loss: 0.26922738552093506, Accuracy: 90.30999755859375, Test Loss: 0.2748110890388489, Test Accuracy: 89.77000427246094
Epoch 3, Loss: 0.22510425746440887, Accuracy: 91.71666717529297, Test Loss: 0.25917163491249084, Test Accuracy: 90.58999633789062
Epoch 4, Loss: 0.19310012459754944, Accuracy: 92.78333282470703, Test Loss: 0.2640419602394104, Test Accuracy: 90.75
Epoch 5, Loss: 0.1685289591550827, Accuracy: 93.80833435058594, Test Loss: 0.25385957956314087, Test Accuracy: 90.94000244140625


In [9]:
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  640       
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  692288    
                                                                 
 dense_1 (Dense)             multiple                  650       
                                                                 
Total params: 693,578
Trainable params: 693,578
Non-trainable params: 0
____________________________________________________